In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)  # for reproducibility

2.3.1


In [2]:
xy = np.loadtxt('data-04-zoo.csv', delimiter = ',', dtype = np.float32)
# 동물의 특징을 추출하여 어떤 종류인지 추측하는 것
    # numpy활용해 데이터를 로드함, ','로 데이터를 나눔
x_data = xy[:, 0:-1]
y_data = xy[:, -1]
    # x_data는 마지막 열을 빼고 가져온다
    # y_data는 마지막 열만 가져온다.

nb_classes = 7  # 0  ~6
    # 종이 7종인지 알 수 있음.

print(y_data)
# Make Y data as onehot shape
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)
    # y값을 보면 onehot으로 이루어져 있지 않음 그래서 tf.one_hot()함수를 사용하여 원핫인코딩을 해줌
    # 7개의 클래스와 y로 원핫을 해줌
    # n차원에서 n+1차원으로, 한 차원이 추가됨
print(Y_one_hot)

print(x_data.shape, Y_one_hot.shape)

[0. 0. 3. 0. 0. 0. 0. 3. 3. 0. 0. 1. 3. 6. 6. 6. 1. 0. 3. 0. 1. 1. 0. 1.
 5. 4. 4. 0. 0. 0. 5. 0. 0. 1. 3. 0. 0. 1. 3. 5. 5. 1. 5. 1. 0. 0. 6. 0.
 0. 0. 0. 5. 4. 6. 0. 0. 1. 1. 1. 1. 3. 3. 2. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 6. 3. 0. 0. 2. 6. 1. 1. 2. 6. 3. 1. 0. 6. 3. 1. 5. 4. 2. 2. 3. 0. 0. 1.
 0. 5. 0. 6. 1.]
tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 

In [6]:
W = tf.Variable(tf.random.normal((16, nb_classes)), name = 'weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name = 'bias')
variables = [W, b]

def logit_fn(X):
    return tf.matmul(X, W) + b

# 학습을 하면서 정확도를 맞추기 위한 것으로 사용됨
def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true = Y, y_pred = logits,
                                                      from_logits=True)
        # 인자값으로 logits가 들어감
    cost = tf.reduce_mean(cost_i)
    return cost

def grad_fn(X, Y):
    with tf.GradientTape()  as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
            # 경사하강법 사용
        return grads

# 얼마나 목표에 잘 도달하고 있나 확인하는 함수
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
        # tf.keras.losses.categorical_crossentropy보다 간편하게 softmax함수로 가장 높은 값을 구할 
        # 수 있음. 그래서 hypothesis()함수를 사용하는 것임.
    correct_preditcion = tf.equal(pred, tf.argmax(Y,1))
        # 실제 Y 값과 pred값이 같은지 계산
    accuracy = tf.reduce_mean(tf.cast(correct_preditcion, tf.float32))
        # 얼마나 맞았는지에 대한 평균을 구해 accuracy로 내보낸다.
    return accuracy

In [7]:
def fit(X, Y, epochs = 1000, verbose = 100):
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)
        # 최적화 하는 방법
    
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if((i==0) | ((i+1) % verbose ==0)):
            acc = prediction(X, Y).numpy()
                # 정확도를 구해서 출력함.
            loss = cost_fn(X, Y).numpy()
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))
           
fit(x_data, Y_one_hot)

Steps: 1 Loss: 5.610101222991943, Acc: 0.1683168262243271
Steps: 100 Loss: 0.6019306182861328, Acc: 0.8415841460227966
Steps: 200 Loss: 0.38190871477127075, Acc: 0.9108911156654358
Steps: 300 Loss: 0.2877010405063629, Acc: 0.9405940771102905
Steps: 400 Loss: 0.23101474344730377, Acc: 0.9504950642585754
Steps: 500 Loss: 0.19246312975883484, Acc: 0.9603960514068604
Steps: 600 Loss: 0.16442003846168518, Acc: 0.9603960514068604
Steps: 700 Loss: 0.14311203360557556, Acc: 0.9603960514068604
Steps: 800 Loss: 0.12642519176006317, Acc: 0.9603960514068604
Steps: 900 Loss: 0.11306186765432358, Acc: 0.9900990128517151
Steps: 1000 Loss: 0.10216663777828217, Acc: 0.9900990128517151
